In [1]:
!pip install tensorboard
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 20.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 46.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 70.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 26.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upg

In [2]:
from tqdm import tqdm
from datetime import datetime
import numpy as np
import random
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import os

In [3]:
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [4]:
class VggNet(torch.nn.Module):
    def __init__(self, num_classes=1000, vgg=16):
        """
        VGGNet implementation for image classification.

        Args:
            num_classes (int, optional): Number of output classes. Default is 1000 (for ImageNet).
            vgg (int, optional): VGG configuration, either 11, 13, 16 or 19 for VGG-11, VGG-16 or VGG-19. 
                                    Default is 19.
        """
        super(VggNet, self).__init__()
        self.num_classes = num_classes

        if vgg not in (11, 13, 16, 19):
            raise ValueError("vgg must be 11, 13, 16, or 19")

        # Define the number of convolutional layers per block based on the VGG variant.
        # Canonical configurations:
        # VGG-11: [1, 1, 2, 2, 2]
        # VGG-13: [2, 2, 2, 2, 2]
        # VGG-16: [2, 2, 3, 3, 3]
        # VGG-19: [2, 2, 4, 4, 4]
        if vgg == 11:
            conv_counts = [1, 1, 2, 2, 2]
        elif vgg == 13:
            conv_counts = [2, 2, 2, 2, 2]
        elif vgg == 16:
            conv_counts = [2, 2, 3, 3, 3]
        else:  # vgg == 19
            conv_counts = [2, 2, 4, 4, 4]

        # Build convolutional blocks 
        self.block1 = self._create_conv_block(in_channels=3,   out_channels=64,  num_convs=conv_counts[0])
        self.block2 = self._create_conv_block(in_channels=64,  out_channels=128, num_convs=conv_counts[1])
        self.block3 = self._create_conv_block(in_channels=128, out_channels=256, num_convs=conv_counts[2])
        self.block4 = self._create_conv_block(in_channels=256, out_channels=512, num_convs=conv_counts[3])
        self.block5 = self._create_conv_block(in_channels=512, out_channels=512, num_convs=conv_counts[4])

        # Fully connected classifier for training mode (after flattening)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # for 224x224 input images
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
        
        # Save fully connected layers to transfer weights later on
        self.fc1 = self.classifier[0]
        self.fc2 = self.classifier[3]
        self.fc3 = self.classifier[6]

        # Convolutional classifier for evaluation
        self.conv_classifier = nn.Sequential(
            nn.Conv2d(512, 4096, kernel_size=7),
            nn.ReLU(inplace=True),
            nn.Conv2d(4096, 4096, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(4096, num_classes, kernel_size=1),
            nn.AdaptiveAvgPool2d(1)
        )

        # Weight initialization recursively to all submodules
        self.apply(self._initialize_weights)


    def _create_conv_block(self, in_channels, out_channels, num_convs):
        """
        Create a convolutional block as:
        [num_convs x (Conv2d -> ReLU)] -> MaxPool2d

        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            num_convs (int): Number of convolutional layers in the block.
        Returns:
            nn.Sequential: The convolutional block.
        """
        layers = []
        for _ in range(num_convs):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
            layers.append(nn.ReLU(inplace=True))
            in_channels = out_channels  # the next convolution uses out_channels as input

        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        return nn.Sequential(*layers)


    def _initialize_weights(self, module):
        """
        Initialize the weights using Xavier uniform initialization.
        """
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)

    
    def fc2conv_weights(self):
        """
        Convert the fully connected classifier weights to convolutional weights for dense evaluation.
        """
        with torch.no_grad():
            # First FC layer to first conv layer
            self.conv_classifier[0].weight.copy_(self.fc1.weight.view(4096, 512, 7, 7))
            self.conv_classifier[0].bias.copy_(self.fc1.bias)

            # Second FC layer to third conv layer
            self.conv_classifier[2].weight.copy_(self.fc2.weight.view(4096, 4096, 1, 1))
            self.conv_classifier[2].bias.copy_(self.fc2.bias)
            
            # Third FC layer to fifth conv layer
            self.conv_classifier[4].weight.copy_(self.fc3.weight.view(self.fc3.out_features, 4096, 1, 1))
            self.conv_classifier[4].bias.copy_(self.fc3.bias)

    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        
        if self.training:
            x = torch.flatten(x, 1)
            x = self.classifier(x)
        else:
            x = self.conv_classifier(x)
            # Reshape (n_batch, n_classes, 1, 1) to (n_batch, n_classes) 
            x = x.squeeze(-1).squeeze(-1) 

        return x

In [5]:
# transpose is use to meet the shape of the tensor C x H x W rather than H x W x C
mean_pixels = np.float32(np.load('./mean_pixels.npy').transpose(2, 0, 1))

def meanSubstraction(x):
    return x - mean_pixels

def toTensorNoScaling(x):
    return torch.from_numpy(np.array(x).transpose(2, 0, 1))

def scaleJitter(x, min_size=256, max_size=512):
    S = random.randint(min_size, max_size)
    return transforms.Resize(S, antialias=True)(x)

def multiResize(x):
    resized_images = [transforms.Resize(size, antialias=True)(x) for size in [256, 384, 512]]
    return resized_images

def PCAColorAugmentation(img, std=0.1, eps=1e-6):
    # Convert PIL image to numpy array
    img = np.array(img)
    img_shape = img.shape  # Original shape (H, W, C)
    # Flatten to (H*W, 3) for RGB channels
    img_flat = img.reshape(-1, 3)
    
    # Compute covariance matrix between channels (3x3)
    img_cov = np.cov(img_flat, rowvar=False)
    # Get eigenvalues and eigenvectors (sorted in ascending order)
    eig_vals, eig_vecs = np.linalg.eigh(img_cov)
    eig_vals = eig_vals[::-1]  # Descending order
    eig_vecs = eig_vecs[:, ::-1]
    eig_vals = np.maximum(eig_vals, eps)
    # Generate random alphas from N(0, std)
    alpha = np.random.normal(0, std, 3)
    
    # Compute correction using sqrt of eigenvalues (standard deviations)
    correction = eig_vecs @ (alpha * np.sqrt(eig_vals))
    # Reshape correction for broadcasting (1, 3) over all pixels
    correction = correction.reshape(1, 3)
    # Apply correction and clip values
    new_img_flat = img_flat + correction
    new_img_flat = np.clip(new_img_flat, 0, 255).astype(np.float32)
    # Reshape to original dimensions, transpose to CxHxW and put between [0-1]
    new_img = new_img_flat.reshape(img_shape).transpose(2, 0, 1)
    
    return torch.from_numpy(new_img)  # Shape (C, H, W)

# Image transformation
transform_train = transforms.Compose([
    # Apply PCA color transformation
    transforms.Lambda(PCAColorAugmentation),
    # Remove mean
    transforms.Lambda(meanSubstraction),
    # Scale jitter between 256 and 512
    transforms.Lambda(scaleJitter),
    # 224x224 random crop 
    transforms.RandomCrop(224),
    # Horizontal reflection with p=0.5
    transforms.RandomHorizontalFlip(p=0.5),
])

transform_val = transforms.Compose([
    # Transform to tensor without scaling
    transforms.Lambda(toTensorNoScaling),
    # Remove mean
    transforms.Lambda(meanSubstraction),
    # Resize to 256, 384 and 512
    transforms.Lambda(multiResize),
])

In [6]:
def train_one_epoch(epoch_index, writer, model, training_dataloader, optimizer, loss_fn, device, scaler, log_batches=1000):
    """
    Trains VGG for one epoch, tracking loss, top-1 and top-5 error rates.
    
    Args:
        epoch_index (int): Current epoch number.
        writer (SummaryWriter): TensorBoard writer object.
        model (VggNet): VGG model instance.
        training_dataloader (DataLoader): PyTorch dataloader for training data.
        optimizer (torch.optim.Optimizer): Optimizer instance.
        loss_fn (nn.Module): Loss function.
        device (torch.device): Device to train on (cuda/cpu).
    
    Returns:
        tuple: (average_loss, average_top1_error, average_top5_error) for the epoch.
    """
    total_loss = 0.0
    running_loss = 0.0
    total_top1_error = 0.0
    total_top5_error = 0.0
    running_top1_error = 0.0
    running_top5_error = 0.0
    
    # Ensure model is in training mode
    model.train()
    
    for i, data in enumerate(tqdm(training_dataloader)):
        inputs, labels = data
        inputs = inputs.to(device, memory_format=torch.channels_last)
        labels = labels.to(device)
        
        # Forward pass under mixed precision context
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Calculate top-1 and top-5 error rates
        _, top5_preds = outputs.topk(5, dim=1, largest=True, sorted=True)
        correct_top1 = top5_preds[:, 0] == labels
        correct_top5 = top5_preds.eq(labels.view(-1, 1)).sum(dim=1) > 0
        top1_error = 1 - correct_top1.sum().item() / labels.size(0)
        top5_error = 1 - correct_top5.sum().item() / labels.size(0)
        
        # Update running statistics
        running_loss += loss.item()
        running_top1_error += top1_error
        running_top5_error += top5_error

        # Update total statistics
        total_loss += loss.item() 
        total_top1_error += top1_error
        total_top5_error += top5_error
        
        # Log every 1000 batches
        if i % log_batches == (log_batches-1):
            avg_running_loss = running_loss / log_batches
            avg_top1_error = 100.0 * running_top1_error / log_batches
            avg_top5_error = 100.0 * running_top5_error / log_batches
                        
            # Log to TensorBoard
            tb_x = epoch_index * len(training_dataloader) + i + 1
            writer.add_scalar('Loss/train_step', avg_running_loss, tb_x)
            writer.add_scalar('Top-1 error/train_step', avg_top1_error, tb_x)
            writer.add_scalar('Top-5 error/train_step', avg_top5_error, tb_x)

            print(f'  Batch {tb_x} Loss: {avg_running_loss:.4f} Top-1 error rate: {avg_top1_error:.2f}% Top-5 error rate: {avg_top5_error:.2f}%')
            
            running_loss = 0.0
            running_top1_error = 0.0
            running_top5_error = 0.0
    
    # Calculate epoch-level metrics
    avg_epoch_loss = total_loss / len(training_dataloader)
    avg_top1_error = 100.0 * total_top1_error / len(training_dataloader)
    avg_top5_error = 100.0 * total_top5_error / len(training_dataloader)
    
    return avg_epoch_loss, avg_top1_error, avg_top5_error

In [7]:
def validate_one_epoch(epoch_index, writer, model, validation_dataloader, loss_fn, device):
    """
    Validates VGG for one epoch, tracking loss, top-1 and top-5 error rates.
    
    Args:
        epoch_index (int): Current epoch number.
        writer (SummaryWriter): TensorBoard writer object.
        model (VggNet): VGG model instance.
        validation_dataloader (DataLoader): PyTorch dataloader for validation data.
        loss_fn (nn.Module): Loss function.
        device (torch.device): Device to run validation on (cuda/cpu).
    
    Returns:
        tuple: (average_loss, average_top1_error, average_top5_error) for the epoch.
    """
    total_loss = 0.0
    total_top1_error = 0.0
    total_top5_error = 0.0
    running_loss = 0.0
    running_top1_error = 0.0
    running_top5_error = 0.0
    
    # Ensure model is in evaluation mode
    model.eval()
    # Transfer fully connected weights to convolutional layers
    model.fc2conv_weights()
    
    with torch.no_grad():  # Disable gradient computation
        for i, data in enumerate(tqdm(validation_dataloader, desc='Validation')):
            inputs, labels = data
            # Move each crop tensor in the inputs tuple to the device
            inputs = [inp.to(device, memory_format=torch.channels_last) for inp in inputs]
            labels = labels.to(device)

            with torch.cuda.amp.autocast():
                outputs = []
                for inp in inputs:
                    out = model(inp)
                    outputs.append(out)
        
                # Forward pass through the model
                outputs = torch.stack(outputs)     # Shape (3, batch, num_classes)
                avg_outputs = outputs.mean(dim=0)  # Shape (batch, num_classes)
                
                # Calculate loss using averaged outputs
                loss = loss_fn(avg_outputs, labels)
            
            # Calculate top-1 and top-5 error rates
            _, top5_preds = avg_outputs.topk(5, dim=1, largest=True, sorted=True)
            correct_top1 = top5_preds[:, 0] == labels
            correct_top5 = top5_preds.eq(labels.view(-1, 1)).any(dim=1)
            top1_error = 1 - correct_top1.float().mean().item()
            top5_error = 1 - correct_top5.float().mean().item()
            
            # Update running statistics
            running_loss += loss.item()
            running_top1_error += top1_error
            running_top5_error += top5_error

            # Update total statistics
            total_loss += loss.item()
            total_top1_error += top1_error
            total_top5_error += top5_error
            
            # Log every 100 batches
            if i % 100 == 99:
                avg_running_loss = running_loss / 100
                avg_top1_error = 100.0 * running_top1_error / 100
                avg_top5_error = 100.0 * running_top5_error / 100
                
                print(f'  Validation Batch {i + 1:5d} Loss: {avg_running_loss:.4f} Top-1 error: {avg_top1_error:.2f}% Top-5 error: {avg_top5_error:.2f}%')
                
                # Log to TensorBoard
                tb_x = epoch_index * len(validation_dataloader) + i + 1
                writer.add_scalar('Loss/val_step', avg_running_loss, tb_x)
                writer.add_scalar('Top-1 error/val_step', avg_top1_error, tb_x)
                writer.add_scalar('Top-5 error/val_step', avg_top5_error, tb_x)
                
                # Reset running statistics
                running_loss = 0.0
                running_top1_error = 0.0
                running_top5_error = 0.0
    
    # Calculate epoch-level metrics
    avg_epoch_loss = total_loss / len(validation_dataloader)
    avg_top1_error = 100.0 * total_top1_error / len(validation_dataloader)
    avg_top5_error = 100.0 * total_top5_error / len(validation_dataloader)
    
    return avg_epoch_loss, avg_top1_error, avg_top5_error

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create model
vggnet = VggNet(num_classes=1000, vgg=16)
vggnet.to(device, memory_format=torch.channels_last)
# Compile model
compiled_vggnet = torch.compile(vggnet)

In [9]:
filepath = '../imagenet'
print('Loading data...')
training_data = torchvision.datasets.ImageNet(filepath, split='train', transform=transform_train)
validation_data = torchvision.datasets.ImageNet(filepath, split='val', transform=transform_val)

training_dataloader = torch.utils.data.DataLoader(
    training_data,
    batch_size=256,  
    shuffle=True,
    num_workers=16,  
    prefetch_factor=4,  
    persistent_workers=True,
    pin_memory=True,
)

validation_dataloader = torch.utils.data.DataLoader(
    validation_data,
    batch_size=128,  # I will get 3 out of each
    shuffle=False,
    num_workers=8,
    prefetch_factor=4,
    persistent_workers=True,
    pin_memory=True
)
print('Finished loading')

Loading data...
Finished loading


In [10]:
writer = SummaryWriter('runs/vgg')

loss_fn   = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vggnet.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
# Initialize the gradient scaler
scaler    = torch.cuda.amp.GradScaler()    

EPOCHS = 100
epoch_number = 0
best_vloss = 1000000.0
for epoch in range(EPOCHS):
    print(f'EPOCH {epoch_number + 1}:')

    # Training phase
    avg_train_loss, avg_train_top1_error, avg_train_top5_error = train_one_epoch(
        epoch_number,
        writer,
        compiled_vggnet,
        training_dataloader,
        optimizer,
        loss_fn,
        device,
        scaler,
    )

    # Validation phase
    avg_val_loss, avg_val_top1_error, avg_val_top5_error = validate_one_epoch(
        epoch_number,
        writer,
        compiled_vggnet,
        validation_dataloader,
        loss_fn,
        device
    )

    print(f'LOSS train {avg_train_loss:.4f} valid {avg_val_loss:.4f}')
    print(f'Top-1 Error train {avg_train_top1_error:.2f}% val {avg_val_top1_error:.2f}%')
    print(f'Top-5 Error train {avg_train_top5_error:.2f}% val {avg_val_top5_error:.2f}%')

    # Update scheduler
    scheduler.step()

    # Log epoch-level metrics
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training': avg_train_loss, 'Validation': avg_val_loss },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Top-1 Error',
                    { 'Training': avg_train_top1_error, 'Validation': avg_val_top1_error },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Top-5 Error',
                    { 'Training': avg_train_top5_error, 'Validation': avg_val_top5_error },
                    epoch_number + 1)
    writer.add_scalar('Learning rate', scheduler.get_last_lr()[0], epoch_number + 1)
    writer.flush()

    # Track best performance and save model
    if avg_val_loss < best_vloss:
        best_vloss = avg_val_loss
        model_path = f'models/model_{epoch_number + 1}.pth' 
        torch.save(vggnet.state_dict(), model_path)  # Save model

    epoch_number += 1

EPOCH 1:


 20%|█▉        | 1000/5005 [04:51<16:57,  3.94it/s]

  Batch 1000 Loss: 6.8782 Top-1 error rate: 99.86% Top-5 error rate: 99.29%


 40%|███▉      | 2000/5005 [09:05<12:45,  3.92it/s]

  Batch 2000 Loss: 6.8314 Top-1 error rate: 99.77% Top-5 error rate: 98.94%


...

  Validation Batch   200 Loss: 1.1252 Top-1 error: 27.11% Top-5 error: 8.86%


Validation:  77%|███████▋  | 300/391 [02:13<00:39,  2.33it/s]

  Validation Batch   300 Loss: 1.4110 Top-1 error: 32.59% Top-5 error: 12.42%


Validation: 100%|██████████| 391/391 [02:59<00:00,  2.18it/s]

LOSS train 1.3405 valid 1.1878
Top-1 Error train 32.81% val 28.33%
Top-5 Error train 12.93% val 9.66%
